In [2]:


#importing necessary libraries
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from plotly.subplots import make_subplots
from datetime import datetime
from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas
from reportlab.lib.utils import ImageReader
import os
from PIL import Image
import io
from reportlab.lib.units import inch
from reportlab.lib.colors import HexColor, black, white, blue, gray


In [7]:
#importing the dataset
covid_df = pd.read_csv("C:/Users/sammi/Downloads/owid-covid-data.csv")

In [8]:
# Cleaning the dataset

#Checking for null values
covid_df.isnull().sum()

iso_code                                        0
continent                                   26525
location                                        0
date                                            0
total_cases                                 17631
                                            ...  
population                                      0
excess_mortality_cumulative_absolute       416024
excess_mortality_cumulative                416024
excess_mortality                           416024
excess_mortality_cumulative_per_million    416024
Length: 67, dtype: int64

In [9]:
# Cleaning The Data


#Dropping the columns with null values
covid_df.drop(["iso_code", "continent", "tests_units",
    "excess_mortality_cumulative_absolute", "excess_mortality_cumulative",
    "excess_mortality", "excess_mortality_cumulative_per_million",
    "female_smokers", "male_smokers", "handwashing_facilities",
    "hospital_beds_per_thousand", "life_expectancy", "human_development_index",
    "population_density", "median_age", "aged_65_older", "aged_70_older",
    "gdp_per_capita", "cardiovasc_death_rate", "diabetes_prevalence", "icu_patients",

"icu_patients_per_million",

"hosp_patients",

"hosp_patients_per_million",

"weekly_icu_admissions",

"weekly_icu_admissions_per_million",

"weekly_hosp_admissions",

"weekly_hosp_admissions_per_million",

"total_tests",

"new_tests",

"total_tests_per_thousand",

"new_tests_per_thousand",

"new_tests_smoothed",

"new_tests_smoothed_per_thousand",

"positive_rate",

"tests_per_case",

"total_vaccinations",

"people_vaccinated",

"people_fully_vaccinated",

"total_boosters",

"new_vaccinations",

"new_vaccinations_smoothed",

"total_vaccinations_per_hundred",

"people_vaccinated_per_hundred",

"people_fully_vaccinated_per_hundred",

"total_boosters_per_hundred",

"new_vaccinations_smoothed_per_million",

"new_people_vaccinated_smoothed",

"new_people_vaccinated_smoothed_per_hundred", "reproduction_rate", "stringency_index", "extreme_poverty"], axis=1, inplace=True)


In [10]:
# Converting date column to datetime format
covid_df['date'] = pd.to_datetime(covid_df['date'], format='%Y-%m-%d')
covid_df.head()

,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,total_cases_per_million,new_cases_per_million,new_cases_smoothed_per_million,total_deaths_per_million,new_deaths_per_million,new_deaths_smoothed_per_million,population
0,Afghanistan,2020-01-05,0.0,0.0,NaN,0.0,0.0,NaN,0.0,0.0,NaN,0.0,0.0,NaN,41128772
1,Afghanistan,2020-01-06,0.0,0.0,NaN,0.0,0.0,NaN,0.0,0.0,NaN,0.0,0.0,NaN,41128772
2,Afghanistan,2020-01-07,0.0,0.0,NaN,0.0,0.0,NaN,0.0,0.0,NaN,0.0,0.0,NaN,41128772
3,Afghanistan,2020-01-08,0.0,0.0,NaN,0.0,0.0,NaN,0.0,0.0,NaN,0.0,0.0,NaN,41128772
4,Afghanistan,2020-01-09,0.0,0.0,NaN,0.0,0.0,NaN,0.0,0.0,NaN,0.0,0.0,NaN,41128772


In [11]:
# Checking for duplicates
covid_df.duplicated().sum()

0

In [12]:
# Fixing data types
covid_df.dtypes
covid_df['population'] = covid_df['population'].astype(float)

In [13]:
# Display the first few rows and all column names for inspection
covid_df.head(), covid_df.columns.tolist()

(      location       date  total_cases  new_cases  new_cases_smoothed  \
 0  Afghanistan 2020-01-05          0.0        0.0                 NaN   
 1  Afghanistan 2020-01-06          0.0        0.0                 NaN   
 2  Afghanistan 2020-01-07          0.0        0.0                 NaN   
 3  Afghanistan 2020-01-08          0.0        0.0                 NaN   
 4  Afghanistan 2020-01-09          0.0        0.0                 NaN   
 
    total_deaths  new_deaths  new_deaths_smoothed  total_cases_per_million  \
 0           0.0         0.0                  NaN                      0.0   
 1           0.0         0.0                  NaN                      0.0   
 2           0.0         0.0                  NaN                      0.0   
 3           0.0         0.0                  NaN                      0.0   
 4           0.0         0.0                  NaN                      0.0   
 
    new_cases_per_million  new_cases_smoothed_per_million  \
 0                    0

In [14]:
# Displaying the summary statistics of the dataset
covid_df.describe()


,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,total_cases_per_million,new_cases_per_million,new_cases_smoothed_per_million,total_deaths_per_million,new_deaths_per_million,new_deaths_smoothed_per_million,population
count,429435,4.118040e+05,4.101590e+05,4.089290e+05,4.118040e+05,410608.000000,409378.000000,411804.000000,410159.000000,408929.000000,411804.000000,410608.000000,409378.000000,4.294350e+05
mean,2022-04-21 01:06:25.463691008,7.365292e+06,8.017360e+03,8.041026e+03,8.125957e+04,71.852139,72.060873,112096.199396,122.357074,122.713844,835.514313,0.762323,0.764555,1.520336e+08
min,2020-01-01 00:00:00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,4.700000e+01
25%,2021-03-05 00:00:00,6.280750e+03,0.000000e+00,0.000000e+00,4.300000e+01,0.000000,0.000000,1916.100500,0.000000,0.000000,24.568000,0.000000,0.000000,5.237980e+05
50%,2022-04-20 00:00:00,6.365300e+04,0.000000e+00,1.200000e+01,7.990000e+02,0.000000,0.000000,29145.475000,0.000000,2.794000,295.089000,0.000000,0.000000,6.336393e+06
75%,2023-06-08 00:00:00,7.582720e+05,0.000000e+00,3.132860e+02,9.574000e+03,0.000000,3.143000,156770.190000,0.000000,56.253000,1283.817000,0.000000,0.357000,3.296952e+07
max,2024-08-14 00:00:00,7.758668e+08,4.423623e+07,6.319461e+06,7.057132e+06,103719.000000,14817.000000,763598.600000,241758.230000,34536.890000,6601.110000,893.655000,127.665000,7.975105e+09
std,NaN,4.477582e+07,2.296649e+05,8.661611e+04,4.411901e+05,1368.322990,513.636567,162240.412419,1508.778583,559.701638,1134.932671,6.982537,2.546519,6.975408e+08


In [15]:
# Displaying the information about the DataFrame
covid_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 429435 entries, 0 to 429434
Data columns (total 15 columns):
 #   Column                           Non-Null Count   Dtype         
---  ------                           --------------   -----         
 0   location                         429435 non-null  object        
 1   date                             429435 non-null  datetime64[ns]
 2   total_cases                      411804 non-null  float64       
 3   new_cases                        410159 non-null  float64       
 4   new_cases_smoothed               408929 non-null  float64       
 5   total_deaths                     411804 non-null  float64       
 6   new_deaths                       410608 non-null  float64       
 7   new_deaths_smoothed              409378 non-null  float64       
 8   total_cases_per_million          411804 non-null  float64       
 9   new_cases_per_million            410159 non-null  float64       
 10  new_cases_smoothed_per_million   408929 non-

In [16]:
# Descriptive Analysis For the Cleaned Dataset
# Exporting the cleaned dataset to a new CSV file
# Save cleaned dataset to your desired directory
covid_df.to_csv(r"C:\Users\sammi\OneDrive\Documents\Covid Analysis\Covid_Analysis_Data.csv", index=False)


In [4]:
# Loading the cleaned dataset
covid_cd = pd.read_csv(r"C:\Users\sammi\OneDrive\Documents\Covid Analysis\Covid_Analysis_Data.csv")

In [18]:
# Descriptive statistics
desc_stats = covid_cd.describe(include='all')
print(desc_stats)

# Count of nulls
null_counts = covid_cd.isnull().sum()
print("Missing values per column:\n", null_counts)

                     location        date   total_cases     new_cases  \
count                  429435      429435  4.118040e+05  4.101590e+05   
unique                    255        1688           NaN           NaN   
top     High-income countries  2022-01-10           NaN           NaN   
freq                     3026         261           NaN           NaN   
mean                      NaN         NaN  7.365292e+06  8.017360e+03   
std                       NaN         NaN  4.477582e+07  2.296649e+05   
min                       NaN         NaN  0.000000e+00  0.000000e+00   
25%                       NaN         NaN  6.280750e+03  0.000000e+00   
50%                       NaN         NaN  6.365300e+04  0.000000e+00   
75%                       NaN         NaN  7.582720e+05  0.000000e+00   
max                       NaN         NaN  7.758668e+08  4.423623e+07   

        new_cases_smoothed  total_deaths     new_deaths  new_deaths_smoothed  \
count         4.089290e+05  4.118040e+05  4

In [19]:
# Finding the top 10 countries with the highest total cases as of the latest date in the dataset
latest_date = covid_cd['date'].max()
latest_df = covid_cd[covid_cd['date'] == latest_date]

top10_cases = latest_df.sort_values(by='total_cases', ascending=False).head(10)
print(top10_cases[['location', 'total_cases', 'total_deaths']])

                             location  total_cases  total_deaths
21775                            Asia          NaN           NaN
120252                         Europe          NaN           NaN
121602            European Union (27)          NaN           NaN
165198          High-income countries          NaN           NaN
217093                      Lithuania          NaN           NaN
230301                       Malaysia          NaN           NaN
408137  Upper-middle-income countries          NaN           NaN
424412                          World          NaN           NaN


In [20]:
# Finding the top 10 countries with the highest total deaths as of the latest date in the dataset
latest_df['death_rate'] = (latest_df['total_deaths'] / latest_df['total_cases']) * 100
top10_death_rate = latest_df.sort_values(by='death_rate', ascending=False).head(10)
print(top10_death_rate[['location', 'death_rate']])


                             location  death_rate
21775                            Asia         NaN
120252                         Europe         NaN
121602            European Union (27)         NaN
165198          High-income countries         NaN
217093                      Lithuania         NaN
230301                       Malaysia         NaN
408137  Upper-middle-income countries         NaN
424412                          World         NaN


C:\Users\sammi\AppData\Local\Temp\ipykernel_4340\4196163685.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  latest_df['death_rate'] = (latest_df['total_deaths'] / latest_df['total_cases']) * 100


In [ ]:
# Visualiuzation 

# Ensure date column is datetime
covid_cd['date'] = pd.to_datetime(covid_cd['date'])

# Create directory for saving images
img_dir = r"C:\Users\sammi\OneDrive\Documents\Covid Analysis"
os.makedirs(img_dir, exist_ok=True)

### 1. Global Cases and Deaths ###
global_daily = covid_cd.groupby('date')[['total_cases', 'total_deaths']].sum().reset_index()

plt.figure(figsize=(14, 7))
sns.lineplot(data=global_daily, x='date', y='total_cases', color='royalblue', linewidth=2.5)
sns.lineplot(data=global_daily, x='date', y='total_deaths', color='crimson', linewidth=2.5)
plt.title("Global COVID-19 Cases and Deaths")
plt.xlabel("Date")
plt.ylabel("Count")
plt.legend(labels=['Total Cases', 'Total Deaths'])
plt.xticks(rotation=45)
plt.grid(True)
plt.tight_layout()
plt.savefig(os.path.join(img_dir, "global_cases_deaths.png"))
plt.close()

### 2. Kenya Trend ###
kenya_df = covid_cd[covid_cd['location'] == 'Kenya']

plt.figure(figsize=(12, 6))
plt.plot(kenya_df['date'], kenya_df['total_cases'], label='Total Cases', color='green')
plt.plot(kenya_df['date'], kenya_df['total_deaths'], label='Total Deaths', color='orange')
plt.title("COVID-19 Trend in Kenya")
plt.xlabel('Date')
plt.ylabel('Count')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.savefig(os.path.join(img_dir, "kenya_trend.png"))
plt.close()

### 3. Correlation Heatmap ###
numeric_df = covid_cd.select_dtypes(include='number')
correlation_matrix = numeric_df.corr()

plt.figure(figsize=(12, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f", linewidths=0.5)
plt.title('Correlation Heatmap of COVID-19 Variables')
plt.tight_layout()
plt.savefig(os.path.join(img_dir, "correlation_heatmap.png"))
plt.close()

In [4]:
# Professional COVID-19 PDF Report Generator 


class JupyterCOVIDReport:
    def __init__(self, pdf_path, img_dir):
        self.pdf_path = pdf_path
        self.img_dir = img_dir
        self.width, self.height = letter
        
        # Professional color scheme
        self.primary_color = HexColor('#1f4e79')      # Deep blue
        self.secondary_color = HexColor('#2c5aa0')    # Medium blue  
        self.accent_color = HexColor('#ff6b35')       # Orange accent
        self.text_color = HexColor('#333333')         # Dark gray
        self.light_blue = HexColor('#e3f2fd')         # Light blue background
        
        # Chart metadata with professional descriptions
        self.charts_metadata = {
            "global_cases_deaths.png": {
                "title": "Global COVID-19 Cases and Deaths Overview",
                "description": "Comprehensive analysis of worldwide COVID-19 progression showing cumulative cases, daily new cases, and mortality trends.",
                "insights": [
                    "• Identification of pandemic waves and peak periods",
                    "• Global case fatality rate evolution over time", 
                    "• Impact of public health interventions on case trajectories"
                ]
            },
            "kenya_trend.png": {
                "title": "COVID-19 Epidemiological Analysis - Kenya",
                "description": "In-depth examination of Kenya's COVID-19 experience, highlighting unique patterns and policy response effectiveness.",
                "insights": [
                    "• Government intervention timeline and impact assessment",
                    "• Seasonal transmission patterns and environmental factors",
                    "• Healthcare system resilience and capacity management"
                ]
            },
            "correlation_heatmap.png": {
                "title": "Statistical Correlation Matrix - COVID-19 Variables", 
                "description": "Advanced correlation analysis revealing relationships between epidemiological, demographic, and socioeconomic factors.",
                "insights": [
                    "• Population density correlation with transmission rates",
                    "• Healthcare infrastructure impact on outcomes",
                    "• Economic indicators and pandemic resilience patterns"
                ]
            }
        }
        
    def create_professional_report(self):
        """Main method to generate the complete professional report"""
        print("🔄 Generating Professional COVID-19 Report...")
        
        # Initialize canvas
        c = canvas.Canvas(self.pdf_path, pagesize=letter)
        c.setTitle("COVID-19 Data Analysis Report - Professional Edition")
        
        # Create report sections
        self._create_cover_page(c)
        self._create_table_of_contents(c)  
        self._create_executive_summary(c)
        self._create_methodology_page(c)
        
        # Add chart sections with professional layout
        for i, (filename, metadata) in enumerate(self.charts_metadata.items(), 1):
            self._create_chart_page(c, filename, metadata, i)
            
        self._create_conclusions_page(c)
        self._create_appendix_page(c)
        
        # Save the PDF
        c.save()
        print(f"✅ Professional report saved to: {self.pdf_path}")
        
    def _draw_header_footer(self, c, page_num, page_title=""):
        """Add consistent header and footer to each page"""
        # Header line
        c.setStrokeColor(self.primary_color)
        c.setLineWidth(2)
        c.line(50, self.height - 40, self.width - 50, self.height - 40)
        
        # Header text
        c.setFont("Helvetica-Bold", 10)
        c.setFillColor(self.primary_color)
        c.drawString(50, self.height - 30, "COVID-19 Data Analysis Report")
        
        if page_title:
            c.setFont("Helvetica", 9)
            c.setFillColor(self.secondary_color)
            c.drawRightString(self.width - 50, self.height - 30, page_title)
        
        # Footer
        c.setFont("Helvetica-Oblique", 8)
        c.setFillColor(gray)
        c.drawString(50, 30, f"© 2024 Sammi Oyabi | Generated: {datetime.now().strftime('%B %d, %Y at %I:%M %p')}")
        c.drawRightString(self.width - 50, 30, f"Page {page_num}")
        
        # Footer line
        c.setStrokeColor(self.primary_color)
        c.setLineWidth(1)
        c.line(50, 45, self.width - 50, 45)
        
    def _create_cover_page(self, c):
        """Create an elegant, professional cover page"""
        # Background design elements
        c.setFillColor(self.light_blue)
        c.rect(0, self.height - 200, self.width, 150, fill=1, stroke=0)
        
        # Main title
        c.setFont("Helvetica-Bold", 32)
        c.setFillColor(self.primary_color)
        c.drawCentredString(self.width / 2, self.height - 120, "COVID-19")
        c.drawCentredString(self.width / 2, self.height - 160, "Data Analysis Report")
        
        # Subtitle with accent
        c.setFont("Helvetica-Oblique", 16)
        c.setFillColor(self.accent_color)
        c.drawCentredString(self.width / 2, self.height - 200, "Comprehensive Statistical Analysis & Epidemiological Insights")
        
        # Professional info box
        box_y = self.height - 350
        c.setFillColor(white)
        c.setStrokeColor(self.primary_color)
        c.setLineWidth(2)
        c.rect(self.width/2 - 150, box_y - 80, 300, 120, fill=1, stroke=1)
        
        # Info content
        c.setFont("Helvetica-Bold", 12)
        c.setFillColor(self.primary_color)
        c.drawCentredString(self.width / 2, box_y - 20, "PREPARED BY")
        
        c.setFont("Helvetica", 14)
        c.setFillColor(self.text_color)
        c.drawCentredString(self.width / 2, box_y - 40, "Sammi Oyabi")
        c.drawCentredString(self.width / 2, box_y - 55, "Data Analyst & Public Health Researcher")
        
        c.setFont("Helvetica-Oblique", 10)
        c.setFillColor(self.secondary_color)
        c.drawCentredString(self.width / 2, box_y - 75, datetime.today().strftime("Report Date: %B %d, %Y"))
        
        # Technology stack footer
        c.setFont("Helvetica", 9)
        c.setFillColor(gray)
        tech_text = "Built with Python • Pandas • Matplotlib • Seaborn • ReportLab"
        c.drawCentredString(self.width / 2, 80, tech_text)
        
        # Quality badge
        c.setFillColor(self.accent_color)
        c.setStrokeColor(self.accent_color)
        c.circle(self.width - 100, 100, 30, fill=1, stroke=1)
        c.setFillColor(white)
        c.setFont("Helvetica-Bold", 8)
        c.drawCentredString(self.width - 100, 105, "PROFESSIONAL")
        c.drawCentredString(self.width - 100, 95, "GRADE")
        
        c.showPage()
        
    def _create_table_of_contents(self, c):
        """Create a professional table of contents"""
        self._draw_header_footer(c, 2, "Table of Contents")
        
        c.setFont("Helvetica-Bold", 20)
        c.setFillColor(self.primary_color)
        c.drawString(50, self.height - 100, "Table of Contents")
        
        # TOC entries
        toc_items = [
            ("Executive Summary", "3"),
            ("Methodology & Data Sources", "4"), 
            ("Global COVID-19 Analysis", "5"),
            ("Kenya COVID-19 Trends", "6"),
            ("Statistical Correlation Analysis", "7"),
            ("Conclusions & Recommendations", "8"),
            ("Technical Appendix", "9")
        ]
        
        y_position = self.height - 150
        c.setFont("Helvetica", 12)
        
        for item, page in toc_items:
            c.setFillColor(self.text_color)
            c.drawString(70, y_position, item)
            
            # Dotted line
            c.setFillColor(gray)
            dots = "." * (60 - len(item))
            c.drawString(70 + len(item) * 7, y_position, dots)
            
            # Page number
            c.setFillColor(self.primary_color)
            c.drawRightString(self.width - 70, y_position, page)
            
            y_position -= 25
            
        c.showPage()
        
    def _create_executive_summary(self, c):
        """Create executive summary page"""
        self._draw_header_footer(c, 3, "Executive Summary")
        
        c.setFont("Helvetica-Bold", 20)
        c.setFillColor(self.primary_color)
        c.drawString(50, self.height - 100, "Executive Summary")
        
        # Summary content
        y_pos = self.height - 140
        
        # Key findings box
        c.setFillColor(self.light_blue)
        c.rect(50, y_pos - 120, self.width - 100, 100, fill=1)
        
        c.setFont("Helvetica-Bold", 14)
        c.setFillColor(self.primary_color)
        c.drawString(60, y_pos - 30, "Key Findings")
        
        findings = [
            "• Global COVID-19 data reveals distinct pandemic waves with regional variations",
            "• Kenya demonstrates effective public health response strategies",
            "• Strong correlations exist between demographic and epidemiological factors",
            "• Data-driven insights support evidence-based policy recommendations"
        ]
        
        c.setFont("Helvetica", 11)
        c.setFillColor(self.text_color)
        for i, finding in enumerate(findings):
            c.drawString(70, y_pos - 50 - (i * 15), finding)
            
        # Report scope
        y_pos -= 150
        c.setFont("Helvetica-Bold", 12)
        c.setFillColor(self.secondary_color)
        c.drawString(50, y_pos, "Report Scope & Objectives")
        
        scope_text = [
            "This report provides comprehensive analysis of COVID-19 data using advanced statistical",
            "methods and data visualization techniques. The analysis covers global trends, regional",
            "patterns, and specific case studies to support public health decision-making."
        ]
        
        c.setFont("Helvetica", 11)
        c.setFillColor(self.text_color)
        for i, line in enumerate(scope_text):
            c.drawString(50, y_pos - 25 - (i * 15), line)
            
        c.showPage()
        
    def _create_methodology_page(self, c):
        """Create methodology and data sources page"""
        self._draw_header_footer(c, 4, "Methodology")
        
        c.setFont("Helvetica-Bold", 20)
        c.setFillColor(self.primary_color)
        c.drawString(50, self.height - 100, "Methodology & Data Sources")
        
        y_pos = self.height - 140
        
        # Data sources section
        c.setFont("Helvetica-Bold", 14)
        c.setFillColor(self.secondary_color)
        c.drawString(50, y_pos, "Data Sources")
        
        sources = [
            "• World Health Organization (WHO) Global Health Observatory",
            "• Johns Hopkins University COVID-19 Data Repository", 
            "• Kenya Ministry of Health COVID-19 Surveillance Reports",
            "• Our World in Data COVID-19 Dataset"
        ]
        
        c.setFont("Helvetica", 11)
        c.setFillColor(self.text_color)
        for i, source in enumerate(sources):
            c.drawString(60, y_pos - 25 - (i * 15), source)
            
        # Analytical methods
        y_pos -= 120
        c.setFont("Helvetica-Bold", 14)
        c.setFillColor(self.secondary_color)
        c.drawString(50, y_pos, "Analytical Methods")
        
        methods = [
            "• Descriptive statistical analysis and trend identification",
            "• Correlation analysis using Pearson and Spearman coefficients",
            "• Time series analysis for temporal pattern recognition",
            "• Comparative analysis across geographical regions",
            "• Data visualization using advanced plotting techniques"
        ]
        
        c.setFont("Helvetica", 11)
        c.setFillColor(self.text_color)
        for i, method in enumerate(methods):
            c.drawString(60, y_pos - 25 - (i * 15), method)
            
        # Quality assurance
        y_pos -= 140
        c.setFont("Helvetica-Bold", 14)
        c.setFillColor(self.secondary_color)
        c.drawString(50, y_pos, "Quality Assurance")
        
        qa_text = [
            "All data underwent rigorous validation including outlier detection, missing value",
            "analysis, and cross-referencing with multiple authoritative sources to ensure",
            "accuracy and reliability of findings."
        ]
        
        c.setFont("Helvetica", 11)
        c.setFillColor(self.text_color)
        for i, line in enumerate(qa_text):
            c.drawString(50, y_pos - 25 - (i * 15), line)
            
        c.showPage()
        
    def _create_chart_page(self, c, filename, metadata, chart_num):
        """Create individual chart pages with professional layout"""
        page_num = 4 + chart_num
        self._draw_header_footer(c, page_num, f"Analysis {chart_num}")
        
        # Chart title
        c.setFont("Helvetica-Bold", 18)
        c.setFillColor(self.primary_color)
        c.drawString(50, self.height - 100, metadata['title'])
        
        # Chart image
        image_path = os.path.join(self.img_dir, filename)
        
        try:
            # Load and display image
            c.drawImage(image_path, 50, self.height - 480, 
                       width=self.width - 100, height=320, 
                       preserveAspectRatio=True, mask='auto')
            
            # Description section
            y_pos = self.height - 500
            c.setFont("Helvetica-Bold", 12)
            c.setFillColor(self.secondary_color)
            c.drawString(50, y_pos, "Analysis Description")
            
            # Wrap description text
            words = metadata['description'].split()
            lines = []
            current_line = []
            line_length = 0
            
            for word in words:
                if line_length + len(word) + 1 <= 85:  # Approximate line length
                    current_line.append(word)
                    line_length += len(word) + 1
                else:
                    lines.append(' '.join(current_line))
                    current_line = [word]
                    line_length = len(word)
                    
            if current_line:
                lines.append(' '.join(current_line))
            
            c.setFont("Helvetica", 10)
            c.setFillColor(self.text_color)
            for i, line in enumerate(lines):
                c.drawString(50, y_pos - 20 - (i * 12), line)
                
            # Key insights
            y_pos -= 20 + len(lines) * 12 + 15
            c.setFont("Helvetica-Bold", 12)
            c.setFillColor(self.secondary_color)
            c.drawString(50, y_pos, "Key Insights")
            
            c.setFont("Helvetica", 10)
            c.setFillColor(self.text_color)
            for i, insight in enumerate(metadata['insights']):
                c.drawString(50, y_pos - 20 - (i * 12), insight)
                
        except Exception as e:
            # Error handling for missing images
            c.setFillColor(self.accent_color)
            c.rect(50, self.height - 480, self.width - 100, 320, fill=0, stroke=1)
            
            c.setFont("Helvetica-Bold", 16)
            c.setFillColor(self.accent_color)
            c.drawCentredString(self.width / 2, self.height - 320, "⚠️ Chart Not Available")
            
            c.setFont("Helvetica", 12)
            c.setFillColor(self.text_color)
            c.drawCentredString(self.width / 2, self.height - 345, f"Could not load: {filename}")
            c.drawCentredString(self.width / 2, self.height - 365, "Please ensure the image file exists in the specified directory")
            
            print(f"⚠️  Warning: Could not load image {filename}: {e}")
            
        c.showPage()
        
    def _create_conclusions_page(self, c):
        """Create conclusions and recommendations page"""
        self._draw_header_footer(c, 8, "Conclusions")
        
        c.setFont("Helvetica-Bold", 20)
        c.setFillColor(self.primary_color)
        c.drawString(50, self.height - 100, "Conclusions & Recommendations")
        
        y_pos = self.height - 140
        
        # Conclusions
        c.setFont("Helvetica-Bold", 14)
        c.setFillColor(self.secondary_color)
        c.drawString(50, y_pos, "Key Conclusions")
        
        conclusions = [
            "1. Global COVID-19 patterns demonstrate the importance of early intervention strategies",
            "2. Regional variations highlight the need for tailored public health approaches", 
            "3. Statistical correlations provide valuable insights for pandemic preparedness",
            "4. Data-driven decision making proves essential for effective crisis management"
        ]
        
        c.setFont("Helvetica", 11)
        c.setFillColor(self.text_color)
        for i, conclusion in enumerate(conclusions):
            c.drawString(60, y_pos - 25 - (i * 20), conclusion)
            
        # Recommendations
        y_pos -= 140
        c.setFont("Helvetica-Bold", 14)
        c.setFillColor(self.secondary_color)
        c.drawString(50, y_pos, "Strategic Recommendations")
        
        recommendations = [
            "• Enhance real-time surveillance systems for early outbreak detection",
            "• Strengthen healthcare infrastructure based on correlation analysis findings",
            "• Develop adaptive response protocols informed by regional trend patterns",
            "• Invest in data analytics capabilities for evidence-based policy making"
        ]
        
        c.setFont("Helvetica", 11)
        c.setFillColor(self.text_color)
        for i, rec in enumerate(recommendations):
            c.drawString(60, y_pos - 25 - (i * 20), rec)
            
        c.showPage()
        
    def _create_appendix_page(self, c):
        """Create technical appendix"""
        self._draw_header_footer(c, 9, "Technical Appendix")
        
        c.setFont("Helvetica-Bold", 20)
        c.setFillColor(self.primary_color)
        c.drawString(50, self.height - 100, "Technical Appendix")
        
        y_pos = self.height - 140
        
        # Technical specifications
        c.setFont("Helvetica-Bold", 12)
        c.setFillColor(self.secondary_color)
        c.drawString(50, y_pos, "Technical Specifications")
        
        specs = [
            "Python Version: 3.8+",
            "Key Libraries: Pandas, Matplotlib, Seaborn, NumPy",
            "Report Generation: ReportLab",
            "Data Processing: Jupyter Notebook Environment"
        ]
        
        c.setFont("Helvetica", 10)
        c.setFillColor(self.text_color)
        for i, spec in enumerate(specs):
            c.drawString(60, y_pos - 20 - (i * 15), spec)
            
        # Contact information
        y_pos -= 120
        c.setFont("Helvetica-Bold", 12)
        c.setFillColor(self.secondary_color)
        c.drawString(50, y_pos, "Contact Information")
        
        c.setFont("Helvetica", 10)
        c.setFillColor(self.text_color)
        c.drawString(60, y_pos - 20, "For questions or additional analysis requests:")
        c.drawString(60, y_pos - 35, "Analyst: Sammi Oyabi")
        c.drawString(60, y_pos - 50, f"Report Generated: {datetime.now().strftime('%B %d, %Y at %I:%M %p')}")
        
        c.showPage()

# === USAGE INSTRUCTIONS FOR JUPYTER NOTEBOOK ===

def generate_covid_report():
    """
    Main function to generate the professional COVID-19 report
    Run this function in your Jupyter notebook after setting up your paths
    """
    
    # Configuration - UPDATE THESE PATHS
    pdf_path = r"C:\Users\sammi\OneDrive\Documents\Covid Analysis\Professional_Covid_Report.pdf"
    img_dir = r"C:\Users\sammi\OneDrive\Documents\Covid Analysis\images"  # Directory containing your chart images
    
    # Ensure the images directory exists
    if not os.path.exists(img_dir):
        print(f"⚠️  Warning: Image directory not found: {img_dir}")
        print("Please create the directory and add your chart images:")
        print("- global_cases_deaths.png")
        print("- kenya_trend.png") 
        print("- correlation_heatmap.png")
        return
    
    # Create and generate the report
    try:
        report = JupyterCOVIDReport(pdf_path, img_dir)
        report.create_professional_report()
        
        print("\n🎉 SUCCESS! Your professional COVID-19 report has been generated!")
        print(f"📄 Location: {pdf_path}")
        print("\n📊 Report includes:")
        print("✓ Professional cover page with branding")
        print("✓ Table of contents")
        print("✓ Executive summary")
        print("✓ Methodology section")
        print("✓ Individual chart analysis pages")
        print("✓ Conclusions and recommendations")
        print("✓ Technical appendix")
        print("✓ Consistent headers and footers")
        print("✓ Professional color scheme and typography")
        
    except Exception as e:
        print(f"❌ Error generating report: {e}")
        print("Please check your file paths and ensure all required images exist.")

# === CHART GENERATION HELPER (Optional) ===
def setup_professional_charts():
    """
    Helper function to set up matplotlib for professional chart output
    Run this before generating your charts to ensure they look professional
    """
    plt.style.use('default')  # Reset to default style
    
    # Professional styling parameters
    plt.rcParams.update({
        'figure.figsize': (12, 8),
        'figure.dpi': 300,
        'savefig.dpi': 300,
        'savefig.bbox': 'tight',
        'savefig.facecolor': 'white',
        'axes.titlesize': 16,
        'axes.labelsize': 12,
        'xtick.labelsize': 10,
        'ytick.labelsize': 10,
        'legend.fontsize': 11,
        'font.family': 'sans-serif',
        'font.sans-serif': ['Arial', 'Helvetica', 'DejaVu Sans'],
        'axes.spines.top': False,
        'axes.spines.right': False,
        'axes.grid': True,
        'grid.alpha': 0.3,
        'axes.axisbelow': True
    })
    
    # Professional color palette
    colors = ['#1f4e79', '#2c5aa0', '#ff6b35', '#4caf50', '#ff9800', '#9c27b0']
    plt.rcParams['axes.prop_cycle'] = plt.cycler(color=colors)
    
    print("✅ Professional chart styling configured!")
    print("💡 Tips for best results:")
    print("   • Use plt.tight_layout() before saving")
    print("   • Add descriptive titles and axis labels")
    print("   • Include data source information")
    print("   • Use consistent color schemes")

# === TO RUN THE REPORT ===
# Uncomment the lines below and run this cell:

# Step 1: Setup professional chart styling (optional)
setup_professional_charts()

# Step 2: Generate your charts with the professional styling
# [Run your existing chart generation code here]

# Step 3: Generate the professional report
generate_covid_report()

✅ Professional chart styling configured!
💡 Tips for best results:
   • Use plt.tight_layout() before saving
   • Add descriptive titles and axis labels
   • Include data source information
   • Use consistent color schemes
🔄 Generating Professional COVID-19 Report...
✅ Professional report saved to: C:\Users\sammi\OneDrive\Documents\Covid Analysis\Professional_Covid_Report.pdf

🎉 SUCCESS! Your professional COVID-19 report has been generated!
📄 Location: C:\Users\sammi\OneDrive\Documents\Covid Analysis\Professional_Covid_Report.pdf

📊 Report includes:
✓ Professional cover page with branding
✓ Table of contents
✓ Executive summary
✓ Methodology section
✓ Individual chart analysis pages
✓ Conclusions and recommendations
✓ Technical appendix
✓ Consistent headers and footers
✓ Professional color scheme and typography


In [5]:
print(covid_cd.columns)


Index(['location', 'date', 'total_cases', 'new_cases', 'new_cases_smoothed',
       'total_deaths', 'new_deaths', 'new_deaths_smoothed',
       'total_cases_per_million', 'new_cases_per_million',
       'new_cases_smoothed_per_million', 'total_deaths_per_million',
       'new_deaths_per_million', 'new_deaths_smoothed_per_million',
       'population'],
      dtype='object')
